In [1]:
from PIL import Image,ImageOps
import numpy as np
import pytesseract
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

from matplotlib.pyplot import figure

In [2]:
# """
# Description : OCR 
# """

# All the required functions

def residual_noise_removal(s):
    s=s.replace("aell1","ae11")
    s=s.replace("aell","ae11")
    s=s.replace("ael","ae1")
    s=s.replace("]1","1")
    s=s.replace("]","1")
    s=s.replace("?","2")
    s=s.replace("corel","core1")
    s=s.replace("EHPCOREI1","EHPCORE1")
    s=s.replace("AF10","AE10")
    s=s.replace("BEPCORE 1","BEPCORE1")
    s=s.replace("ae113","ae13")
    s=s.replace("IPREKP1524","IPREKP152A")
    s=s.replace("IPRNNP151 to PRGHP155 BE40(40G)","IPRNNP151 to IPRGHP155 BE40(40G)")
    s=s.replace("IPRMICP155-TEPCORE1:BE15","IPRMCP155-TEPCORE1:BE15")
    s=s.replace("NNPCORE1I to TEPCORE1(ae11 100G)","NNPCORE1 to TEPCORE1(ae11 100G)")
    s=s.replace("GTPCORE1-BEPCORE1 AEF9 : Intra capital Link(100G)","GTPCORE1-BEPCORE1 AE9 : Intra capital Link(100G)")
    s=s.replace("EHPCORE2-AEPCORE2 ae110 : Intracapital Link(20G)","EHPCORE2-AEPCORE2 ae10 : Intracapital Link(20G)")
    s=s.replace("EX-NN Primary Link - IPRNNP152 to IPREXP156 — Bundle-","EX-NN Primary Link - IPRNNP152 to IPREXP156 -- Bundle-")
    s=s.replace("EX-NN Primary Link - IPRNNP152 to IPREXP156 — Bundle-","EX-NN Primary Link - IPRNNP152 to IPREXP156 -- Bundle-")
    s=s.replace("EK-GH Secondary Link - IPREKP152A to IPRGHP156A — Bundle-","EK-GH Secondary Link - IPREKP152A to IPRGHP156A -- Bundle-")
    return s

def remove_noise_and_smooth(img):
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    or_image = cv2.bitwise_or(img, closing)
    return or_image

def textCheck(imgData):
# """
# Input : Image 
# output : text found in image
# """
    ret,img = cv2.threshold(np.array(imgData), 125, 255, cv2.THRESH_BINARY)

    #remove noise
    img=remove_noise_and_smooth(img)
    new_image = Image.fromarray(img.astype(np.uint8))

    # padding and expansion	
    new_image=ImageOps.expand(new_image,(0,0,0,10),(256,256,256))
    new_image = Image.fromarray(np.array(new_image)).convert('L') #now need gray scale

    basewidth = 900

    new_image = new_image.resize((900,47), Image.ANTIALIAS)

    text = pytesseract.image_to_string(new_image, lang = 'eng')

    
    return text.strip()

    
#     text = list(text)
#     for i in range(len(text)):
#         if text[i]==']':
#             text[i]="1"
#         elif text[i]=='?':
#             text[i]="2"
#     text = "".join(text)

In [3]:
# This function takes all the images from a folder and makes a dataframe which contains the name of the file, the output by
# Sunil sir, the output after applying the logic and an empty column which has to be filled manually to check the actual correct output.

def load_images_from_folder(folder):     
    i=0                                  # This is just a counter to make sure the code is running
    images = []                          # Image array in which all the images from the folder are stored
    data = []
    df = pd.DataFrame(data, columns = ['Graph_Name', 'Original Title', 'Simple OCR1', 'OCR_Updated (OCR2)']) 
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        t = textCheck(img)
        df = df.append({'Graph_Name' : filename, 'Simple OCR1' : t,'OCR_Updated (OCR2)' : residual_noise_removal(t)} , ignore_index=True)
        i=i+1
        print(i)
        if img is not None:
            images.append(img)
    return images, df

In [4]:
# You can change this file destination according to the data present in your device

images, df = load_images_from_folder("cropTextImg/cropTextImg")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [5]:
# Exporting the data to the excel sheet

df.to_excel('OCR.xlsx')